# Milestone 1
### By Carlos Camacho
This is a Jupyter Notebook showing an example image of a satellite dataset read in using Rasterio, and demonstration of a function that carries out a manipulation of that image using Keras and Tensorflow 2.0.

# Imports

In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import scipy
from PIL import Image
from tensorflow.keras.preprocessing.image import load_img

D:\apps\Anaconda\envs\liveproject\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\apps\Anaconda\envs\liveproject\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\apps\Anaconda\envs\liveproject\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\apps\Anaconda\envs\liveproject\lib\site-packages\tensorflow\python\framework

In [ ]:
def normalize(array):
    array_min, array_max = array.min(), array.max()
    return (array - array_min) / (array_max - array_min)### Define useful functions
   
def plot_img(dataset):
    b1 = dataset.read(3)
    b2 = dataset.read(2)
    b3 = dataset.read(1)

    b1n = normalize(b1)
    b2n = normalize(b2)
    b3n = normalize(b3)

    comp = np.dstack((b1n, b2n, b3n))
    plt.figure(figsize=(10,10))
    plt.imshow(comp)
    
def plot_histogram(dataset):
    from rasterio.plot import show_hist
    plt.figure(figsize=(10,10))
    show_hist(dataset, bins=50, lw=0.0, stacked=False, alpha=0.3, histtype='stepfilled', title='Histogram')

# Use of rasterio for data manipulation
### Opening the raster dataset

In [ ]:
dataset = rasterio.open('presa_calles.jpg')
print('Raster dataset info:')
print(dataset.profile)

### Plotting the raster dataset and histogram values

In [ ]:
plot_img(dataset)

Histogram of raster data

In [ ]:
plot_histogram(dataset)

### Reprojecting data

In [ ]:
from rasterio.warp import calculate_default_transform, reproject, Resampling

dst_crs = 'EPSG:4326'

transform, width, height = calculate_default_transform(dataset.crs, dst_crs, dataset.width, dataset.height, *dataset.bounds)
kwargs = dataset.meta.copy()
kwargs.update({
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height
})
with rasterio.open('presa_calles_wgs84.jpg', 'w', **kwargs) as output_ds:
    for i in range(1, dataset.count + 1):
        reproject(source=rasterio.band(dataset, i),
                 destination=rasterio.band(output_ds, i),
                 src_transform=dataset.transform,
                 src_crs=dataset.crs,
                 dst_transform=transform,
                 dst_crs=dst_crs,
                 resampling=Resampling.nearest)
dataset_wgs84 = rasterio.open('presa_calles_wgs84.jpg')
print(dataset_wgs84.profile)

In [ ]:
from rasterio.plot import show
plt.figure(figsize=(10,10))
show(dataset_wgs84)
plot_histogram(dataset_wgs84)

# Use of tensorflow for data manipulation

In [ ]:
import matplotlib.image as mpimg
filename = 'presa_calles.jpg'

image = mpimg.imread(filename)
print(image.shape)

plt.figure(figsize=(10,10))
plt.imshow(img)

### Transpose of image

In [ ]:
x = tf.Variable(image, name='x')

model = tf.global_variables_initializer()

with tf.Session() as session:
    x = tf.transpose(x, perm=[1, 0, 2])
    session.run(model)
    result = session.run(x)

print(result.shape)
plt.figure(figsize=(10,10))
plt.imshow(result)

### Reverse of image

In [ ]:
height, width, bands = image.shape
x = tf.Variable(image, name='x')

model = tf.global_variables_initializer()

with tf.Session() as session:
    x = tf.reverse_sequence(x, [width]*height, 1, batch_dim=0)
    session.run(model)
    result = session.run(x)
    
print(result.shape)
plt.figure(figsize=(10,10))
plt.imshow(result)